In [ ]:
!nvidia-smi

In [ ]:
!pip -q install -U ultralytics roboflow albumentations opencv-python tqdm

In [ ]:
from roboflow import Roboflow
from pathlib import Path
import yaml, os

# 🔧 Fill these from your project URL / settings
ROBOFLOW_API_KEY = ""
WORKSPACE = ""
PROJECT   = ""
VERSION   = 1       # None = auto-pick latest; or set an int like 5
EXPORT_FMT = "yolo11"

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
proj = rf.workspace(WORKSPACE).project(PROJECT)

if VERSION is None:
    vers = proj.versions()
    if not vers:
        raise RuntimeError("No versions found in this project. Generate a version in Roboflow first.")
    VERSION = getattr(vers[-1], "version", getattr(vers[-1], "version_number", None))
    print("Using latest version:", VERSION)

dataset = proj.version(VERSION).download(EXPORT_FMT)
INPUT_PATH = Path(dataset.location)
print("Downloaded to:", INPUT_PATH)
print("Contents:", [p.name for p in INPUT_PATH.iterdir()])

# Ensure YOLO's expected key name 'val' exists (some exports use 'valid')
yaml_path = INPUT_PATH/"data.yaml"
cfg = yaml.safe_load(open(yaml_path))
if "valid" in cfg and "val" not in cfg:
    cfg["val"] = cfg["valid"]
    yaml.safe_dump(cfg, open(yaml_path, "w"), sort_keys=False)
print("data.yaml ->\n", open(yaml_path).read())

In [ ]:
import torch, os
from pathlib import Path
from ultralytics import YOLO

# 0) Mount Drive once per session
from google.colab import drive
drive.mount('/content/drive')

# (Optional) disable Weights & Biases auto-logging
os.environ["WANDB_DISABLED"] = "true"

print("CUDA available:", torch.cuda.is_available())

# 1) Choose a folder in Drive
OUTPUT_PATH = "/content/drive/MyDrive/runs_roboflow"   # where to save runs
INPUT_PATH = Path("/content/drive/MyDrive/your_dataset_folder")  # <-- set this!
DATA_YAML = INPUT_PATH / "data.yaml"

# 2) Train to that folder: Ultralytics will create OUTPUT_PATH/name/...
model = YOLO("yolo11m.pt")  # or your last.pt

results = model.train(
    data=str(INPUT_PATH / "data.yaml"),
    epochs=100,
    patience=20,      # stop early if no val improvement for 20 epochs
    project=OUTPUT_PATH,          # <-- write directly to Drive
    imgsz=640,
    save=True,        # always save weights
    save_period=5,    # keep a checkpoint every 5 epochs
    close_mosaic=10,  # turn off mosaic in final 10 epochs
    name="v1_yolo11m"
)